<a href="https://colab.research.google.com/github/Trinanjan123/ml_codes_concepts/blob/master/nlp_dl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [2]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [3]:
train.sample(10)

,id,anchor,target,context,score
15782,e7a400d5bd4896fb,hybrid bearing,sealed bearings,F16,0.50
17063,855a3a3fd8d13a1d,instruction processing,instruction execution phase,G06,0.50
30794,284a955d4cf217fe,slip segment,connecting member,B22,0.25
34911,dca58346c83f614a,verifiable,easily verifiable,H04,0.75
19881,fe6c61bbc0efea07,melanocyte,skin melanocyte,C12,0.75
29722,8c61d10f1b7a41dc,selection logic,block,G10,0.00
34081,76218f396103ad7e,turbine machine,turbine,F01,0.50
17610,be7e14f2affbfd7e,inward travel,traveller,G09,0.00
11164,bfa316df3c4fcb85,elevation view,elevational elevation,B66,0.50
30079,9b1464d2dfacb0c7,shaped engaging,ground engaging component,A41,0.00


In [4]:
train.shape

(36473, 5)

In [5]:
train.describe(include='object')

,id,anchor,target,context
count,36473,36473,36473,36473
unique,36473,733,29340,106
top,37d61fd2272659b1,component composite coating,composition,H01
freq,1,152,24,2186


anchor has a lot of repeated values (we see out of 36473, only 733 is unique). Out of those, 'component composite coating' has the hihest frequency appearing 152 times

In [6]:
# Adding string in the specified format
train['input'] = 'TEXT1: ' + train.context + '; TEXT2: ' + train.target + \
              '; ANC1: ' + train.anchor

In [7]:
train['input'].head()

0    TEXT1: A47; TEXT2: abatement of pollution; ANC...
1    TEXT1: A47; TEXT2: act of abating; ANC1: abate...
2    TEXT1: A47; TEXT2: active catalyst; ANC1: abat...
3    TEXT1: A47; TEXT2: eliminating process; ANC1: ...
4    TEXT1: A47; TEXT2: forest region; ANC1: abatement
Name: input, dtype: object

In [8]:
!pip install transformers[sentencepiece]

In [9]:
!pip install datasets

In [10]:
!pip install sentencepiece

In [11]:
from datasets import Dataset, DatasetDict

ds = Dataset.from_pandas(train)

In [12]:
ds

Dataset({
    features: ['id', 'anchor', 'target', 'context', 'score', 'input'],
    num_rows: 36473
})

In [13]:
# Before tokenization, we should choose what model to use because every model has its own tokenization way
model_nm = 'microsoft/deberta-v3-small'

In [14]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

# AutoTokenizer will create a tokenizer appropriate for a given model
tokz = AutoTokenizer.from_pretrained(model_nm)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:470: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [15]:
def tok_func(x):
  return tokz(x['input'])

tok_ds = ds.map(tok_func, batched=True)

Map:   0%|          | 0/36473 [00:00<?, ? examples/s]

In [16]:
row = tok_ds[0]
row['input'], row['input_ids']

('TEXT1: A47; TEXT2: abatement of pollution; ANC1: abatement',
 [1,
  54453,
  435,
  294,
  336,
  5753,
  346,
  54453,
  445,
  294,
  47284,
  265,
  6435,
  346,
  23702,
  435,
  294,
  47284,
  2])

In [17]:
# Transformers library always assumes that the label column is named 'labels'.
# Hence renaming 'score' column to 'labels'
tok_ds = tok_ds.rename_columns({'score':'labels'})

In [18]:
# Transformers expects metrics to be returned as a dict, since that way the
# trainer knows what labels to use
import numpy as np

def corr(x,y):
  return np.corrcoef(x,y)[0][1]

def corr_d(eval_pred):
  return {'pearson': corr(*eval_pred)}

In [19]:
# Training
from transformers import TrainingArguments, Trainer

bs = 128
epochs = 4
lr = 8e-5

In [20]:
!pip install accelerate -U

In [21]:
args = TrainingArguments('outputs',
                         learning_rate=lr,
                         warmup_ratio=0.1,
                         lr_scheduler_type='cosine',
                         fp16=True,
                         evaluation_strategy="epoch",
                         per_device_train_batch_size=bs,
                         per_device_eval_batch_size=bs*2,
                         num_train_epochs=epochs,
                         weight_decay=0.01,
                         report_to='none')

In [22]:
test['input'] = 'TEXT1: ' + test.context + '; TEXT2: ' + test.target + '; ANC1: ' + test.anchor
eval_ds = Dataset.from_pandas(test).map(tok_func, batched=True)

Map:   0%|          | 0/36 [00:00<?, ? examples/s]

In [25]:
dds = tok_ds.train_test_split(0.25, seed=42)
dds

DatasetDict({
    train: Dataset({
        features: ['id', 'anchor', 'target', 'context', 'labels', 'input', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 27354
    })
    test: Dataset({
        features: ['id', 'anchor', 'target', 'context', 'labels', 'input', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 9119
    })
})

In [26]:
model = AutoModelForSequenceClassification.from_pretrained(model_nm, num_labels=1)
trainer = Trainer(model,
                  args,
                  train_dataset=dds['train'],
                  eval_dataset=dds['test'],
                  tokenizer=tokz,
                  compute_metrics=corr_d)

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-small and are newly initialized: ['classifier.weight', 'classifier.bias', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [27]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a DebertaV2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Pearson
1,No log,0.029042,0.798633
2,No log,0.022179,0.820378
3,0.040400,0.022743,0.830975
4,0.040400,0.023321,0.830831


TrainOutput(global_step=856, training_loss=0.029167483343142217, metrics={'train_runtime': 69.0364, 'train_samples_per_second': 1584.903, 'train_steps_per_second': 12.399, 'total_flos': 717218678299260.0, 'train_loss': 0.029167483343142217, 'epoch': 4.0})

In [28]:
preds = trainer.predict(eval_ds).predictions.astype(float)
preds

array([[ 0.56152344],
       [ 0.74267578],
       [ 0.58935547],
       [ 0.36108398],
       [-0.00708389],
       [ 0.47265625],
       [ 0.54785156],
       [ 0.04772949],
       [ 0.19909668],
       [ 1.07617188],
       [ 0.28442383],
       [ 0.28540039],
       [ 0.74951172],
       [ 0.94970703],
       [ 0.72900391],
       [ 0.53222656],
       [ 0.2956543 ],
       [-0.00813293],
       [ 0.67822266],
       [ 0.38061523],
       [ 0.4765625 ],
       [ 0.26293945],
       [ 0.2199707 ],
       [ 0.26831055],
       [ 0.58154297],
       [ 0.00185108],
       [-0.02349854],
       [-0.0120163 ],
       [-0.01287079],
       [ 0.68017578],
       [ 0.33129883],
       [ 0.10827637],
       [ 0.74316406],
       [ 0.5546875 ],
       [ 0.39819336],
       [ 0.26464844]])

In [29]:
preds = np.clip(preds, 0, 1)
preds

array([[0.56152344],
       [0.74267578],
       [0.58935547],
       [0.36108398],
       [0.        ],
       [0.47265625],
       [0.54785156],
       [0.04772949],
       [0.19909668],
       [1.        ],
       [0.28442383],
       [0.28540039],
       [0.74951172],
       [0.94970703],
       [0.72900391],
       [0.53222656],
       [0.2956543 ],
       [0.        ],
       [0.67822266],
       [0.38061523],
       [0.4765625 ],
       [0.26293945],
       [0.2199707 ],
       [0.26831055],
       [0.58154297],
       [0.00185108],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.68017578],
       [0.33129883],
       [0.10827637],
       [0.74316406],
       [0.5546875 ],
       [0.39819336],
       [0.26464844]])